# Training

In [ ]:
from training import train

train(
    color_space = 'YCrCb',
    pixels_per_cell = 8,
    cells_per_block = 2,
    hog_orientations = 9,
    color_hist_nbins = 16,
    window_shape = (8,8)    #cells
)

# Pipeline

In [ ]:
import cv2
import numpy as np
import pickle
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from scipy.ndimage.measurements import label
from window_search import search_cars
%matplotlib inline


with open("model.p", "rb") as f:
    color_space, pixels_per_cell, cells_per_block, hog_orientations, color_hist_nbins, window_shape, svc, X_scaler = pickle.load(f)
    y_start_stop, cells_per_step = (360,None), (2,2)

scales = [2/3]
colors = [(1,0,0), (0,1,0), (0,0,1), (1,0,1), (0,1,1), (1,1,0)]
heatmap = np.zeros(shape=(720,1280), dtype=np.int)


def draw_bboxes(img, bboxes, color):
    for bbox in bboxes:
        cv2.rectangle(img, tuple(bbox[0]), tuple(bbox[1]), color, 5)

def draw_labeled_bboxes(img, labels):
    for car_number, color in zip(range(1, labels[1]+1), colors):
        nonzero = (labels[0] == car_number).nonzero()
        bbox = ((np.min(nonzero[1]), np.min(nonzero[0])), (np.max(nonzero[1]), np.max(nonzero[0])))
        cv2.rectangle(img, bbox[0], bbox[1], color, 5)

def process(img):
    img = img.astype(np.float32)/255
    
    for scale, color in zip(scales, colors):
        bboxes = search_cars(img, y_start_stop, scale, color_space, pixels_per_cell, cells_per_block, hog_orientations, color_hist_nbins, window_shape, cells_per_step, svc, X_scaler)
        #draw_bboxes(img, bboxes, color)
        for box in bboxes:
            heatmap[box[0,1]:box[1,1], box[0,0]:box[1,0]] += 1
    
    heatmap[heatmap < 1] = 0
    labels = label(heatmap)
    print(labels[1], "cars detected")
    draw_labeled_bboxes(img, labels)

    return (img*255).astype(np.uint8)


# Testing

In [ ]:
plt.imshow(process(mpimg.imread('test_images/test5.jpg')))

# Video

In [ ]:
input_file = "project_video.mp4"
output_file = "../output/project_video.mp4"

clip1 = VideoFileClip(input_file)
clip2 = clip1.fl_image(process)
clip2.write_videofile(output_file, audio=False)

In [ ]:
from IPython.display import HTML
HTML("""<video width="960" height="540" controls><source src=\"{0}\"></video>""".format(output_file))